In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
!pip install datasets
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
tokenizer = AutoTokenizer.from_pretrained("suriya7/bart-finetuned-text-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/bart-finetuned-text-summarization")
dataset = load_dataset("kmfoda/booksum")


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
tokenizer = AutoTokenizer.from_pretrained("suriya7/bart-finetuned-text-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/bart-finetuned-text-summarization")

In [22]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
import re
from rouge_score import rouge_scorer

In [24]:
train_dataset = dataset['train']

In [25]:
# dataset['train'][55]["summary_text"]

In [34]:
def break_in_chunks(text,max_chunk,overlap):
    sentences = re.findall(r'.+?\.', text)
    paras=[]
    curr=""
    i = 0
    while i < len(sentences):
        if len(curr)+len(sentences[i])>max_chunk:
            paras.append(curr)
            curr=""
            
            for j in range(i,0,-1):
                if len(curr)+len(sentences[j])>overlap:
#                     print(i)
                    i=j - 1
#                     print(i)
                    curr=""
                    break
                else:
                    curr+=sentences[j]
        else:
            curr+=sentences[i]
        i+=1
    return paras
def generate_summary(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=1024)
    summary_ids = model.generate(inputs.input_ids, max_length=120, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [35]:
from rouge_score import rouge_scorer

def compute_rouge(preds, labels):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(labels, preds)
    return scores

def custom_loss(scores):
    loss = 1 - scores['rouge1'].fmeasure  # Example: using ROUGE-1 F1 score as loss
    return loss


In [36]:
# import torch

# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# model.train()
# num_epochs = 3  # Number of epochs

# for epoch in range(num_epochs):
#     for chapter_num, data in enumerate(train_dataset):
#         chapter_text = data['chapter']
#         target_summary = data['summary_text']

#         text_in_chunks = break_in_chunks(chapter_text, 1024, 200)
#         generated_summary = ""
#         for para_num, para in enumerate(text_in_chunks):
#             para_summary = generate_summary(para, model, tokenizer)
#             generated_summary += para_summary
#             print(para_num)

#         scores = compute_rouge(generated_summary, target_summary)
#         loss = custom_loss(scores)

#         # Ensure loss is a tensor with gradients
#         if isinstance(loss, float):
#             loss = torch.tensor(loss, requires_grad=True)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         print(f"Epoch: {epoch}, Chapter: {chapter_num}, Loss: {loss.item()}")


In [37]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

# Define your custom dataset

# Training parameters
num_train_epochs = 3
warmup_steps = 500
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
weight_decay = 0.01
gradient_accumulation_steps = 16
learning_rate = 5e-5

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

model.train()

# Training loop
for epoch in range(num_train_epochs):
    for step, data in enumerate(train_dataset):
        chapter_text = data['chapter'] # Assuming data['chapter'] is a list of text
        target_summary = data['summary_text'] # Assuming data['summary_text'] is a list of summaries

        # Break chapter text into chunks
        text_in_chunks = break_in_chunks(chapter_text, 1024, 200)
        generated_summary = ""
        for para_num, para in enumerate(text_in_chunks):
            para_summary = generate_summary(para, model, tokenizer)
            generated_summary += para_summary
#             print(f"Para {para_num} summary generated")

        # Calculate scores and loss
        scores = compute_rouge(generated_summary, target_summary)
        loss = custom_loss(scores)

        # Ensure loss is a tensor with gradients
        if isinstance(loss, float):
            loss = torch.tensor(loss, requires_grad=True)

        # Accumulate gradients
        loss = loss / gradient_accumulation_steps
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        print(f"Epoch: {epoch}, Step: {step}, Loss: {loss.item()}")

    # Save model checkpoint at the end of each epoch if needed
    torch.save(model.state_dict(), f"model_checkpoint_epoch_{epoch}.pt")


Epoch: 0, Step: 0, Loss: 0.040043968707323074
Epoch: 0, Step: 1, Loss: 0.041428569704294205
Epoch: 0, Step: 2, Loss: 0.043364811688661575
Epoch: 0, Step: 3, Loss: 0.044380251318216324
Epoch: 0, Step: 4, Loss: 0.04197416827082634
Epoch: 0, Step: 5, Loss: 0.03407534211874008
Epoch: 0, Step: 6, Loss: 0.037332214415073395
Epoch: 0, Step: 7, Loss: 0.03519093990325928
Epoch: 0, Step: 8, Loss: 0.041363637894392014
Epoch: 0, Step: 9, Loss: 0.0384114570915699
Epoch: 0, Step: 10, Loss: 0.04279279336333275
Epoch: 0, Step: 11, Loss: 0.04256889596581459
Epoch: 0, Step: 12, Loss: 0.040087465196847916
Epoch: 0, Step: 13, Loss: 0.038266703486442566
Epoch: 0, Step: 14, Loss: 0.037853773683309555
Epoch: 0, Step: 15, Loss: 0.03896252438426018
Epoch: 0, Step: 16, Loss: 0.03824334964156151
Epoch: 0, Step: 17, Loss: 0.03757225349545479
Epoch: 0, Step: 18, Loss: 0.040783897042274475
Epoch: 0, Step: 19, Loss: 0.04018878564238548
Epoch: 0, Step: 20, Loss: 0.040126457810401917
Epoch: 0, Step: 21, Loss: 0.038461

KeyboardInterrupt: 

In [38]:
torch.save(model.state_dict(), 'model_weights.pth')

In [39]:
model.load_state_dict(torch.load("/kaggle/working/model_weights.pth"))

<All keys matched successfully>

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def compute_similarity_loss(generated_summary, target_summary):
    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    
    # Combine generated and target summaries into one list for vectorization
    all_texts = [generated_summary, target_summary]
    
    # Convert text to TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_texts)
    
    # Calculate cosine similarity between generated and target summary
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    
    # Calculate loss (1 - cosine similarity to minimize the distance)
    loss = 1 - cosine_sim[0][0]
    
    return loss

In [48]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
import random
import numpy as np

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# Define your custom dataset
# train_dataset = ...

# Training parameters
num_train_epochs = 1
learning_rate = 2e-5
train_batch_size = 1
eval_batch_size = 1
weight_decay = 0.01
gradient_accumulation_steps = 1
warmup_steps = 0  # Adjust if necessary

# Set up optimizer
optimizer = Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-8)

# Set up learning rate scheduler
def lr_lambda(current_step: int) -> float:
    # Example: Linear decay from initial learning rate to 0
    if warmup_steps > 0:
        return min(1.0, current_step / warmup_steps)
    return max(0.0, (num_train_epochs * len(train_dataset) - current_step) / (num_train_epochs * len(train_dataset) - warmup_steps))

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

model.train()

# DataLoader setup
best_metric = float('inf')

# Training loop
for epoch in range(num_train_epochs):
    for step, data in enumerate(train_dataset):
        if step<246:continue
        chapter_text = data['chapter']  # Assuming data['chapter'] is a list of text
        target_summary = data['summary_text']  # Assuming data['summary_text'] is a list of summaries

        # Break chapter text into chunks
        text_in_chunks = break_in_chunks(chapter_text, 1024, 200)
        generated_summary = ""
        for para_num, para in enumerate(text_in_chunks):
            para_summary = generate_summary(para, model, tokenizer)
            generated_summary += para_summary

        # Calculate similarity and loss
        loss = compute_similarity_loss(generated_summary, target_summary)

        # Ensure loss is a tensor with gradients
        if isinstance(loss, float):
            loss = torch.tensor(loss, requires_grad=True)

        # Accumulate gradients
        loss = loss / gradient_accumulation_steps
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Update the learning rate
        scheduler.step()

        print(f"Epoch: {epoch}, Step: {step}, Loss: {loss.item()}")

    # Save model checkpoint at the end of each epoch if needed
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }, f"model_checkpoint_epoch_{epoch}.pt")


Epoch: 0, Step: 246, Loss: 0.7382520941167592
Epoch: 0, Step: 247, Loss: 0.5485279063942392
Epoch: 0, Step: 248, Loss: 0.45260847090448375
Epoch: 0, Step: 249, Loss: 0.4658884060737991
Epoch: 0, Step: 250, Loss: 0.4010585705773093
Epoch: 0, Step: 251, Loss: 0.5174602796664387
Epoch: 0, Step: 252, Loss: 0.5591101852511576
Epoch: 0, Step: 253, Loss: 0.5396128702115648
Epoch: 0, Step: 254, Loss: 0.4339703178678538
Epoch: 0, Step: 255, Loss: 0.5514245902645101
Epoch: 0, Step: 256, Loss: 0.3849138331447506
Epoch: 0, Step: 257, Loss: 0.5459564513374924
Epoch: 0, Step: 258, Loss: 0.5320981069951303
Epoch: 0, Step: 259, Loss: 0.6554604734984093
Epoch: 0, Step: 260, Loss: 0.6033442509280758
Epoch: 0, Step: 261, Loss: 0.4648589125819562
Epoch: 0, Step: 262, Loss: 0.6117345114798023
Epoch: 0, Step: 263, Loss: 0.42754703260644533


KeyboardInterrupt: 

In [50]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
import random
import numpy as np

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# Define your custom dataset
# train_dataset = ...

# Training parameters
num_train_epochs = 1
learning_rate = 2e-5
train_batch_size = 1
eval_batch_size = 1
weight_decay = 0.01
gradient_accumulation_steps = 1
warmup_steps = 0  # Adjust if necessary

# Set up optimizer
optimizer = Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-8)

# Set up learning rate scheduler
def lr_lambda(current_step: int) -> float:
    # Example: Linear decay from initial learning rate to 0
    if warmup_steps > 0:
        return min(1.0, current_step / warmup_steps)
    return max(0.0, (num_train_epochs * len(train_dataset) - current_step) / (num_train_epochs * len(train_dataset) - warmup_steps))

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

model.train()

# DataLoader setup
best_metric = float('inf')

# Training loop
for epoch in range(num_train_epochs):
    for step, data in enumerate(train_dataset):
        if step<250:continue
        chapter_text = data['chapter']  # Assuming data['chapter'] is a list of text
        target_summary = data['summary_text']  # Assuming data['summary_text'] is a list of summaries

        # Break chapter text into chunks
        text_in_chunks = break_in_chunks(chapter_text, 1024, 200)
        generated_summary = ""
        for para_num, para in enumerate(text_in_chunks):
            para_summary = generate_summary(para, model, tokenizer)
            generated_summary += para_summary

        # Calculate similarity and loss
        loss = compute_similarity_loss(generated_summary, target_summary)

        # Ensure loss is a tensor with gradients
        if isinstance(loss, float):
            loss = torch.tensor(loss, requires_grad=True)

        # Accumulate gradients
        loss = loss / gradient_accumulation_steps
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Update the learning rate
        scheduler.step()

        print(f"Epoch: {epoch}, Step: {step}, Loss: {loss.item()}")
        
        if loss.item() < best_metric:
            best_metric = loss.item()
            torch.save({
                'epoch': epoch,
                'step': step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': loss.item()
            }, f"best_model_checkpoint_step.pt")
            print(f"Checkpoint saved at step {step} with loss {loss.item()}")

    

Epoch: 0, Step: 250, Loss: 0.4379529472606636
Checkpoint saved at step 250 with loss 0.4379529472606636
Epoch: 0, Step: 251, Loss: 0.5870994917171573
Epoch: 0, Step: 252, Loss: 0.5919497370311839
Epoch: 0, Step: 253, Loss: 0.5177813212953264
Epoch: 0, Step: 254, Loss: 0.4840912108615425
Epoch: 0, Step: 255, Loss: 0.6290961187931176
Epoch: 0, Step: 256, Loss: 0.3952952975589058
Checkpoint saved at step 256 with loss 0.3952952975589058
Epoch: 0, Step: 257, Loss: 0.5426529180773769
Epoch: 0, Step: 258, Loss: 0.5960198224905336
Epoch: 0, Step: 259, Loss: 0.7058862331895563
Epoch: 0, Step: 260, Loss: 0.6635542967311222
Epoch: 0, Step: 261, Loss: 0.4182054947748669
Epoch: 0, Step: 262, Loss: 0.6251555872101893
Epoch: 0, Step: 263, Loss: 0.46428616480320706
Epoch: 0, Step: 264, Loss: 0.454983228876365
Epoch: 0, Step: 265, Loss: 0.6672830711469677
Epoch: 0, Step: 266, Loss: 0.573855602142
Epoch: 0, Step: 267, Loss: 0.5525194793940946
Epoch: 0, Step: 268, Loss: 0.48769172049609855
Epoch: 0, Ste